In [13]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')


In [15]:
# Make two tables, one for the tweet: date,company, tweet . Other for the stock price: date, ticker, open, high, low, close, volume

# Drop any existing collections to avoid duplicates
try:
    client.drop_database('stock_tweets')
except Exception as e:
    print(f"Error dropping database: {e}")
# Create a new database
db = client['stock_tweets']
db.create_collection('tweets')
db.create_collection('stock_prices')
tweets = db['tweets']
stock_prices = db['stock_prices']

# Insert summy data into both the tables
tweets.insert_many([
    {"date": "2023-01-01", "company": "apple", "tweet": "Apple's new product launch was a success!"},
    {"date": "2023-01-02", "company": "google", "tweet": "Google's stock is on the rise after earnings report."},
    {"date": "2023-01-03", "company": "amazon", "tweet": "Amazon Prime Day sales exceeded expectations!"},
    {"date": "2023-01-03", "company": "amazon", "tweet": "Amazon's new warehouse is set to open next month."},
])

stock_prices.insert_many([
    {"date": "2023-01-01", "ticker": "AAPL", "open": 150, "high": 155, "low": 149, "close": 154, "volume": 1000000},
    {"date": "2023-01-02", "ticker": "GOOGL", "open": 2800, "high": 2850, "low": 2790, "close": 2840, "volume": 500000},
    {"date": "2023-01-03", "ticker": "AMZN", "open": 3300, "high": 3350, "low": 3280, "close": 3305, "volume": 700000}
])


InsertManyResult([ObjectId('68258bedb91dc6e14c228e95'), ObjectId('68258bedb91dc6e14c228e96'), ObjectId('68258bedb91dc6e14c228e97')], acknowledged=True)

In [19]:
for tweet in tweets.find():
    print(tweet)

{'_id': ObjectId('68258bedb91dc6e14c228e91'), 'date': '2023-01-01', 'company': 'apple', 'tweet': "Apple's new product launch was a success!", 'sentiment': 'positive', 'sentiment_score': 0.9026734232902527}
{'_id': ObjectId('68258bedb91dc6e14c228e92'), 'date': '2023-01-02', 'company': 'google', 'tweet': "Google's stock is on the rise after earnings report.", 'sentiment': 'positive', 'sentiment_score': 0.9382569193840027}
{'_id': ObjectId('68258bedb91dc6e14c228e93'), 'date': '2023-01-03', 'company': 'amazon', 'tweet': 'Amazon Prime Day sales exceeded expectations!', 'sentiment': 'positive', 'sentiment_score': 0.9344619512557983}
{'_id': ObjectId('68258bedb91dc6e14c228e94'), 'date': '2023-01-03', 'company': 'amazon', 'tweet': "Amazon's new warehouse is set to open next month.", 'sentiment': 'neutral', 'sentiment_score': 0.9201991558074951}


In [17]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

# Do inference for the tweets and store the sentiment score in the tweets table as an additional column
def get_sentiment(tweet):
    result = pipe(tweet)
    return result[0]['label'], result[0]['score']

for tweet in tweets.find():
    sentiment = get_sentiment(tweet['tweet'])
    tweets.update_one(
        {"_id": tweet['_id']},
        {"$set": {"sentiment": sentiment[0], "sentiment_score": sentiment[1]}}
    )


Device set to use cpu


In [23]:
# Average the sentiment score for each company and date make a new table
avg_sentiment = {}
for tweet in tweets.find():
    date = tweet['date']
    company = tweet['company']
    sentiment_score = tweet['sentiment_score']
    
    if date not in avg_sentiment:
        avg_sentiment[date] = {}
    
    if company not in avg_sentiment[date]:
        avg_sentiment[date][company] = {'total_score': 0, 'count': 0}
    
    avg_sentiment[date][company]['total_score'] += sentiment_score
    avg_sentiment[date][company]['count'] += 1

# Create a new collection for average sentiment
avg_sentiment_collection = db['avg_sentiment']
avg_sentiment_collection.drop()

# Insert average sentiment data into the new collection
for date, companies in avg_sentiment.items():
    for company, data in companies.items():
        avg_score = data['total_score'] / data['count']
        avg_sentiment_collection.insert_one({
            "date": date,
            "company": company,
            "avg_sentiment": avg_score
        })

# Print the average sentiment table
for avg in avg_sentiment_collection.find():
    print(avg)

{'_id': ObjectId('68258d57b91dc6e14c228e98'), 'date': '2023-01-01', 'company': 'apple', 'avg_sentiment': 0.9026734232902527}
{'_id': ObjectId('68258d57b91dc6e14c228e99'), 'date': '2023-01-02', 'company': 'google', 'avg_sentiment': 0.9382569193840027}
{'_id': ObjectId('68258d57b91dc6e14c228e9a'), 'date': '2023-01-03', 'company': 'amazon', 'avg_sentiment': 0.9273305535316467}


In [26]:
company_ticker_mapping = {
    "apple": "AAPL",
    "google": "GOOGL",
    "amazon": "AMZN"
}

# Concatenate the two tables on the date and ticker after converting the company name to ticker
def get_ticker(company):
    return company_ticker_mapping.get(company.lower(), None)

# Create a new collection for the concatenated data
concatenated_collection = db['concatenated_data']
concatenated_collection.drop()
# Insert concatenated data into the new collection
for tweet in tweets.find():
    date = tweet['date']
    company = tweet['company']
    ticker = get_ticker(company)
    
    if ticker:
        stock_data = stock_prices.find_one({"date": date, "ticker": ticker})
        if stock_data:
            concatenated_collection.insert_one({
                "date": date,
                "company": company,
                "tweet": tweet['tweet'],
                "sentiment": tweet['sentiment'],
                "sentiment_score": tweet['sentiment_score'],
                "ticker": ticker,
                **stock_data
            })

# Print the concatenated data
for data in concatenated_collection.find():
    print(data)

DuplicateKeyError: E11000 duplicate key error collection: stock_tweets.concatenated_data index: _id_ dup key: { _id: ObjectId('68258bedb91dc6e14c228e97') }, full error: {'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: stock_tweets.concatenated_data index: _id_ dup key: { _id: ObjectId('68258bedb91dc6e14c228e97') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('68258bedb91dc6e14c228e97')}}